In [11]:
import shutil

shutil.copy2('./../../myCBR_projects/creditt_yedek.prj', './../../myCBR_projects/creditt.prj') # complete target filename given

'./../../myCBR_projects/creditt.prj'

In [1]:
import warnings
import sys

warnings.filterwarnings("ignore")
sys.path.append('../../')
sys.path.append('../')

In [2]:
import random
import pickle
from PertCF import *

In [3]:
# Set random seed
random_state = random.randint(0, 100)

# Concept name
concept = 'credit'


column_names = ['status', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment_duration', 'installment_rate',
       'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'number_credits', 'job',
       'people_liable', 'telephone', 'foreign_worker', 'credit_risk']
train_cols= column_names[:-1]

label = 'credit_risk'

class_names = ['0','1']


num_cols = ['duration','amount','age']
cat_feat = ['status', 'credit_history', 'purpose', 'savings','employment_duration', 
            'installment_rate', 'other_debtors', 'present_residence', 
            'property', 'other_installment_plans', 'housing', 'number_credits', 'job',
            'people_liable', 'telephone', 'foreign_worker']


# Read data
train = pd.read_pickle('./Data/train.pkl')
test =  pd.read_pickle('./Data/test.pkl')


#train[label] = train[label].astype('object')
#test[label] = test[label].astype('string')

for i in num_cols:
    train[i] = train[i].astype('float')
    test[i] = test[i].astype('float')
    
for i in cat_feat:
    train[i] = train[i].astype('object')
    test[i] = test[i].astype('object')

X_train, X_test, y_train, y_test = train[train_cols], test[train_cols], train[[label]], test[[label]]


In [4]:
def label_dict(df, name):
    dic_status = {}
    lst = list(df[name].unique())
    for i in range(len(lst)):
        dic_status[lst[i]] = i 
    return dic_status

def rev(dic):
    return {v: k for k, v in dic.items()}

# from categoric features to numeric features with maps
# from numeric features to categoric features with inv_maps
def encode(df, maps):
    for i in cat_feat:
        df[i] = df[i].map(maps[i]).astype('float')
    return df
    

df = train
maps = {'status':label_dict(df,'status'), 'credit_history':label_dict(df,'credit_history'), 
        'purpose':label_dict(df,'purpose'), 'savings':label_dict(df,'savings'),
        'employment_duration':label_dict(df,'employment_duration'), 
        'installment_rate':label_dict(df,'installment_rate'), 'other_debtors':label_dict(df,'other_debtors'), 
        'present_residence':label_dict(df,'present_residence'), 'property':label_dict(df,'property'), 
        'other_installment_plans':label_dict(df,'other_installment_plans'), 'housing':label_dict(df,'housing'), 
        'number_credits':label_dict(df,'number_credits'), 'job':label_dict(df,'job'),
        'people_liable':label_dict(df,'people_liable'), 'telephone':label_dict(df,'telephone'), 
        'foreign_worker':label_dict(df,'foreign_worker'),'credit_risk':label_dict(df,'credit_risk')}
inv_maps = {}




In [5]:

# encoding for training the model
X_train_en, X_test_en = encode(X_train.copy(), maps), encode(X_test.copy(), maps)
train_en = encode(train.copy(), maps)
test_en = encode(test.copy(), maps)

# load model
model = pickle.load(open('./Model/model_GBC.pkl', "rb"))

# load shap_df
shap_df = pd.read_pickle("./temp_files/shap_df.pkl")

In [6]:
num_iter = 10
explainer = Explainer(model, X_train, X_test, y_train, y_test, label, cat_feat, concept, shap_df=shap_df, maps=maps, inv_maps=inv_maps, num_iter=num_iter)


Given Shap values used.

API connection completed.

Concept names:  ['credit']  Is correct:  True 

cb_class0 added: True
cb_class1 added: True
Casebase names:  [['cb_class1'], ['cb_class0']] 

Cases deleted from concept 'credit': True

Cases imported to the casebases. 

Amalgamation functions created for every class using shap values. 



In [7]:
res_list_names = ['coef', 'num_iter', 'dissimilarity', 'sparsity', 'instability', 'time']
res_list = []


for coef in [1,2,3,4,5,6,7,8,9,10,25,50,75,100,150,200,300,400,500,750]:
    _, res = explainer.testGenerateCF(coef_thresh = coef)
    res_list.append([coef]+[num_iter]+res)

res_df = pd.DataFrame(res_list,columns=res_list_names)

1
2
3
4
5
6
7
8
9
10
25
50
75
100
150
200
300
400
500
750


In [8]:
res_df

,coef,num_iter,dissimilarity,sparsity,instability,time
0,1,10,0.139704,0.704737,0.139803,0.744687
1,2,10,0.202481,0.640000,0.202579,1.581845
2,3,10,0.202481,0.640000,0.202579,2.016430
3,4,10,0.202481,0.640000,0.202579,2.428722
4,5,10,0.202481,0.640000,0.202579,2.947602
5,6,10,0.202481,0.640000,0.202579,3.412218
6,7,10,0.202481,0.640000,0.202579,3.772016
7,8,10,0.202481,0.640000,0.202579,4.187851
8,9,10,0.202481,0.640000,0.202579,4.551142
9,10,10,0.202481,0.640000,0.202579,4.835569


In [9]:
res_df.to_pickle('./Output/exp1_coef_1-750_numIter_10.pkl')

---

In [ ]:
# 2 3 4 5 10 20 50
# 2 3 10 50 20 4 5

In [12]:
shutil.copy2('./../../myCBR_projects/creditt_yedek.prj', './../../myCBR_projects/creditt.prj') # complete target filename given

'./../../myCBR_projects/creditt.prj'

In [1]:
num_iter = 5

In [3]:
import random
import pickle
from PertCF import *

In [4]:
# Set random seed
random_state = random.randint(0, 100)

# Concept name
concept = 'credit'


column_names = ['status', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment_duration', 'installment_rate',
       'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'number_credits', 'job',
       'people_liable', 'telephone', 'foreign_worker', 'credit_risk']
train_cols= column_names[:-1]

label = 'credit_risk'

class_names = ['0','1']


num_cols = ['duration','amount','age']
cat_feat = ['status', 'credit_history', 'purpose', 'savings','employment_duration', 
            'installment_rate', 'other_debtors', 'present_residence', 
            'property', 'other_installment_plans', 'housing', 'number_credits', 'job',
            'people_liable', 'telephone', 'foreign_worker']


# Read data
train = pd.read_pickle('./Data/train.pkl')
test =  pd.read_pickle('./Data/test.pkl')


#train[label] = train[label].astype('object')
#test[label] = test[label].astype('string')

for i in num_cols:
    train[i] = train[i].astype('float')
    test[i] = test[i].astype('float')
    
for i in cat_feat:
    train[i] = train[i].astype('object')
    test[i] = test[i].astype('object')

X_train, X_test, y_train, y_test = train[train_cols], test[train_cols], train[[label]], test[[label]]


In [5]:
def label_dict(df, name):
    dic_status = {}
    lst = list(df[name].unique())
    for i in range(len(lst)):
        dic_status[lst[i]] = i 
    return dic_status

def rev(dic):
    return {v: k for k, v in dic.items()}

# from categoric features to numeric features with maps
# from numeric features to categoric features with inv_maps
def encode(df, maps):
    for i in cat_feat:
        df[i] = df[i].map(maps[i]).astype('float')
    return df
    

df = train
maps = {'status':label_dict(df,'status'), 'credit_history':label_dict(df,'credit_history'), 
        'purpose':label_dict(df,'purpose'), 'savings':label_dict(df,'savings'),
        'employment_duration':label_dict(df,'employment_duration'), 
        'installment_rate':label_dict(df,'installment_rate'), 'other_debtors':label_dict(df,'other_debtors'), 
        'present_residence':label_dict(df,'present_residence'), 'property':label_dict(df,'property'), 
        'other_installment_plans':label_dict(df,'other_installment_plans'), 'housing':label_dict(df,'housing'), 
        'number_credits':label_dict(df,'number_credits'), 'job':label_dict(df,'job'),
        'people_liable':label_dict(df,'people_liable'), 'telephone':label_dict(df,'telephone'), 
        'foreign_worker':label_dict(df,'foreign_worker'),'credit_risk':label_dict(df,'credit_risk')}
inv_maps = {}

In [6]:
# encoding for training the model
#X_en = encode(X.copy(), maps)
X_train_en, X_test_en = encode(X_train.copy(), maps), encode(X_test.copy(), maps)
train_en = encode(train.copy(), maps)
test_en = encode(test.copy(), maps)

# load model
model = pickle.load(open('./Model/model_GBC.pkl', "rb"))

# load shap_df
shap_df = pd.read_pickle("./temp_files/shap_df.pkl")

In [7]:
explainer = Explainer(model, X_train, X_test, y_train, y_test, label, cat_feat, concept, shap_df=shap_df, maps=maps, inv_maps=inv_maps, num_iter=num_iter)

Given Shap values used.

API connection completed.

Concept names:  ['credit']  Is correct:  True 

cb_class0 added: True
cb_class1 added: True
Casebase names:  [['cb_class1'], ['cb_class0']] 

Cases deleted from concept 'credit': True

Cases imported to the casebases. 

Amalgamation functions created for every class using shap values. 



In [8]:
res_list_names = ['coef', 'num_iter', 'dissimilarity', 'sparsity', 'instability', 'time']
res_list = []

for coef in [1,2,3,4,5,6,7,8,9,10,25,50,75,100,150,200,300,400,500,750]:
    _, res = explainer.testGenerateCF(coef_thresh = coef)
    res_list.append([coef]+[num_iter]+res)

res_df = pd.DataFrame(res_list,columns=res_list_names)

1
2
3
4
5
6
7
8
9
10
25
50
75
100
150
200
300
400
500
750


In [9]:
res_df

,coef,num_iter,dissimilarity,sparsity,instability,time
0,1,5,0.140738,0.702632,0.140837,0.267796
1,2,5,0.140740,0.702632,0.140838,0.510631
2,3,5,0.140740,0.702632,0.140838,0.637926
3,4,5,0.140740,0.702632,0.140838,0.786722
4,5,5,0.140740,0.702632,0.140838,0.956485
5,6,5,0.140740,0.702632,0.140838,1.065722
6,7,5,0.140740,0.702632,0.140838,1.195711
7,8,5,0.140740,0.702632,0.140838,1.341544
8,9,5,0.140740,0.702632,0.140838,1.490353
9,10,5,0.140740,0.702632,0.140838,1.625799


In [9]:
res_df

,coef,num_iter,dissimilarity,sparsity,instability,time
0,1,4,0.140760,0.702632,0.140858,0.214493
1,2,4,0.200904,0.640526,0.201002,0.375537
2,3,4,0.200904,0.640526,0.201002,0.486754
3,4,4,0.200904,0.640526,0.201002,0.600318
4,5,4,0.200904,0.640526,0.201002,0.708860
5,6,4,0.200904,0.640526,0.201002,0.821253
6,7,4,0.200904,0.640526,0.201002,0.932536
7,8,4,0.200904,0.640526,0.201002,0.989803
8,9,4,0.200904,0.640526,0.201002,1.096479
9,10,4,0.200904,0.640526,0.201002,1.217921


In [10]:
res_df.to_pickle('./Output/exp1_coef_1-750_numIter_'+str(num_iter)+'.pkl')

---